In [2]:
import gensim, logging

In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
gmodel = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

2023-03-21 22:51:07,384 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin


FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

In [4]:
gmodel['cat']

array([ 0.0123291 ,  0.20410156, -0.28515625,  0.21679688,  0.11816406,
        0.08300781,  0.04980469, -0.00952148,  0.22070312, -0.12597656,
        0.08056641, -0.5859375 , -0.00445557, -0.296875  , -0.01312256,
       -0.08349609,  0.05053711,  0.15136719, -0.44921875, -0.0135498 ,
        0.21484375, -0.14746094,  0.22460938, -0.125     , -0.09716797,
        0.24902344, -0.2890625 ,  0.36523438,  0.41210938, -0.0859375 ,
       -0.07861328, -0.19726562, -0.09082031, -0.14160156, -0.10253906,
        0.13085938, -0.00346375,  0.07226562,  0.04418945,  0.34570312,
        0.07470703, -0.11230469,  0.06738281,  0.11230469,  0.01977539,
       -0.12353516,  0.20996094, -0.07226562, -0.02783203,  0.05541992,
       -0.33398438,  0.08544922,  0.34375   ,  0.13964844,  0.04931641,
       -0.13476562,  0.16308594, -0.37304688,  0.39648438,  0.10693359,
        0.22167969,  0.21289062, -0.08984375,  0.20703125,  0.08935547,
       -0.08251953,  0.05957031,  0.10205078, -0.19238281, -0.09

In [5]:
gmodel['dog']

array([  5.12695312e-02,  -2.23388672e-02,  -1.72851562e-01,
         1.61132812e-01,  -8.44726562e-02,   5.73730469e-02,
         5.85937500e-02,  -8.25195312e-02,  -1.53808594e-02,
        -6.34765625e-02,   1.79687500e-01,  -4.23828125e-01,
        -2.25830078e-02,  -1.66015625e-01,  -2.51464844e-02,
         1.07421875e-01,  -1.99218750e-01,   1.59179688e-01,
        -1.87500000e-01,  -1.20117188e-01,   1.55273438e-01,
        -9.91210938e-02,   1.42578125e-01,  -1.64062500e-01,
        -8.93554688e-02,   2.00195312e-01,  -1.49414062e-01,
         3.20312500e-01,   3.28125000e-01,   2.44140625e-02,
        -9.71679688e-02,  -8.20312500e-02,  -3.63769531e-02,
        -8.59375000e-02,  -9.86328125e-02,   7.78198242e-03,
        -1.34277344e-02,   5.27343750e-02,   1.48437500e-01,
         3.33984375e-01,   1.66015625e-02,  -2.12890625e-01,
        -1.50756836e-02,   5.24902344e-02,  -1.07421875e-01,
        -8.88671875e-02,   2.49023438e-01,  -7.03125000e-02,
        -1.59912109e-02,

In [6]:
gmodel['spatula']

array([-0.19140625, -0.04296875,  0.27539062,  0.00488281, -0.3203125 ,
        0.08203125,  0.05566406, -0.03613281, -0.31445312,  0.10693359,
       -0.359375  ,  0.29882812,  0.02331543,  0.05517578, -0.140625  ,
        0.1953125 , -0.23632812, -0.22167969, -0.06542969, -0.3359375 ,
        0.25195312, -0.09326172,  0.54296875,  0.11328125, -0.28710938,
       -0.12011719, -0.11181641,  0.20996094, -0.33203125,  0.30273438,
       -0.3359375 , -0.12255859,  0.12890625, -0.28515625, -0.04223633,
        0.25585938,  0.3203125 ,  0.07177734,  0.19042969, -0.01379395,
        0.16992188, -0.22460938,  0.5078125 ,  0.08398438, -0.07519531,
       -0.06396484,  0.05371094,  0.34570312,  0.46289062, -0.16699219,
       -0.30664062,  0.15234375, -0.09765625, -0.26171875, -0.14160156,
        0.2265625 ,  0.49609375, -0.10791016, -0.08447266,  0.234375  ,
        0.04931641, -0.07128906,  0.05273438, -0.11914062,  0.09814453,
        0.11181641, -0.13574219, -0.46875   ,  0.26171875,  0.12

In [7]:
gmodel.similarity('cat', 'dog')

0.76094570897822089

In [8]:
gmodel.similarity('cat', 'spatula')

0.12412612600429632

In [6]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [7]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) # strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remove apostrophes
    sent = re.sub(r'\W', ' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated spaces
    sent = sent.strip()
    return sent.split()

In [8]:
# unsupervised training data
import re
import os
unsup_sentences = []

# source: http://ai.stanford.edu/~amaas/data/sentiment/, data from IMDB
for dirname in ["train/pos", "train/neg", "train/unsup", "test/pos", "test/neg"]:
    for fname in sorted(os.listdir("aclImdb/" + dirname)):
        if fname[-4:] == '.txt':
            with open("aclImdb/" + dirname + "/" + fname, encoding='UTF-8') as f:
                sent = f.read()
                words = extract_words(sent)
                unsup_sentences.append(TaggedDocument(words, [dirname + "/" + fname]))

# source: http://www.cs.cornell.edu/people/pabo/movie-review-data/
for dirname in ["review_polarity/txt_sentoken/pos", "review_polarity/txt_sentoken/neg"]:
    for fname in sorted(os.listdir(dirname)):
        if fname[-4:] == '.txt':
            with open(dirname + "/" + fname, encoding='UTF-8') as f:
                for i, sent in enumerate(f):
                    words = extract_words(sent)
                    unsup_sentences.append(TaggedDocument(words, ["%s/%s-%d" % (dirname, fname, i)]))
                
# source: https://nlp.stanford.edu/sentiment/, data from Rotten Tomatoes
with open("stanfordSentimentTreebank/original_rt_snippets.txt", encoding='UTF-8') as f:
    for i, line in enumerate(f):
        words = extract_words(sent)
        unsup_sentences.append(TaggedDocument(words, ["rt-%d" % i]))

In [9]:
len(unsup_sentences)

175325

In [10]:
unsup_sentences[0:10]

[TaggedDocument(words=['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'hig', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', 'the', 'scramble', 'to', 'survive', 'financially', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', 'i', 'immediately', 'recalled', 'at', 'high', 'a', 'classic', 'line', 'inspector', 'here', 'to', 'sack', 'one', 'of', 'your', 'teachers', 'student', 'welcome', 

In [11]:
import random
class PermuteSentences(object):
    def __init__(self, sents):
        self.sents = sents
        
    def __iter__(self):
        shuffled = list(self.sents)
        random.shuffle(shuffled)
        for sent in shuffled:
            yield sent

In [13]:
permuter = PermuteSentences(unsup_sentences) 
model = Doc2Vec(permuter, dm=0, hs=1, vector_size=50)

2023-03-21 23:23:57,205 : INFO : collecting all words and their counts
2023-03-21 23:23:57,479 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-03-21 23:23:57,996 : INFO : PROGRESS: at example #10000, processed 1396420 words (2709469 words/s), 44843 word types, 10000 tags
2023-03-21 23:23:58,623 : INFO : PROGRESS: at example #20000, processed 2815670 words (2267209 words/s), 61122 word types, 20000 tags
2023-03-21 23:23:59,103 : INFO : PROGRESS: at example #30000, processed 4227152 words (2950961 words/s), 72720 word types, 30000 tags
2023-03-21 23:23:59,728 : INFO : PROGRESS: at example #40000, processed 5595829 words (2189628 words/s), 82255 word types, 40000 tags
2023-03-21 23:24:00,244 : INFO : PROGRESS: at example #50000, processed 6980354 words (2693028 words/s), 90372 word types, 50000 tags
2023-03-21 23:24:00,819 : INFO : PROGRESS: at example #60000, processed 8407655 words (2488768 words/s), 97845 word types, 60000 tags
2023-03-21 23:2

In [14]:
# done with training, free up some memory
model.delete_temporary_training_data(keep_inference=True)

AttributeError: 'Doc2Vec' object has no attribute 'delete_temporary_training_data'

In [15]:
model.save('reviews.d2v')
# in other program, we could write: model = Doc2Vec.load('reviews.d2v')

2023-03-21 23:33:48,084 : INFO : Doc2Vec lifecycle event {'fname_or_handle': 'reviews.d2v', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-03-21T23:33:48.084180', 'gensim': '4.3.1', 'python': '3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2023-03-21 23:33:48,091 : INFO : not storing attribute cum_table
2023-03-21 23:33:50,113 : INFO : saved reviews.d2v


In [16]:
model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))

array([ 0.15839541,  0.02213576,  0.26376584, -0.48066616, -0.03137077,
       -0.25015157,  0.26324415, -0.13032252,  0.01211821,  0.24326684,
       -0.16611272, -0.17989054, -0.33873653, -0.07376021,  0.62292314,
        0.35614946, -0.16246454,  0.08911735,  0.01923273,  0.33431724,
        0.44906926,  0.04037865,  0.06475811,  0.2848666 ,  0.32015496,
        0.49359483, -0.31922275, -0.01975652,  0.5144004 , -0.14519018,
       -0.20464548,  0.09204558,  0.28979242,  0.11151716, -0.26435006,
        0.09226024,  0.5630968 ,  0.11117744,  0.6047081 ,  0.09140925,
       -0.07361961,  0.1250164 , -0.3000361 , -0.15348944, -0.08545798,
       -0.44060743, -0.05337591, -0.09558962,  0.1370779 , -0.0928385 ],
      dtype=float32)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))],
    [model.infer_vector(extract_words("Service sucks."))])

array([[0.39466345]], dtype=float32)

In [18]:
cosine_similarity(
    [model.infer_vector(extract_words("Highly recommended."))],
    [model.infer_vector(extract_words("Service sucks."))])

array([[0.21989854]], dtype=float32)

In [20]:
sentences = []
sentvecs = []
sentiments = []
for fname in ["yelp", "amazon_cells", "imdb"]: 
    with open("sentiment labelled sentences/%s_labelled.txt" % fname, encoding='UTF-8') as f:
        for i, line in enumerate(f):
            line_split = line.strip().split('\t')
            sentences.append(line_split[0])
            words = extract_words(line_split[0])
            sentvecs.append(model.infer_vector(words, epochs =10)) # create a vector for this document
            sentiments.append(int(line_split[1]))
            
# shuffle sentences, sentvecs, sentiments together
combined = list(zip(sentences, sentvecs, sentiments))
random.shuffle(combined)
sentences, sentvecs, sentiments = zip(*combined)

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

clf = KNeighborsClassifier(n_neighbors=9)
clfrf = RandomForestClassifier()

In [22]:

#here it takes only 0,6s to compute
scores = cross_val_score(clf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.7886666666666666, 0.018147543451754924)

In [23]:
#this one takes almost 15s

scores = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.792, 0.014583095236151563)

In [24]:
# bag-of-words comparison
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), RandomForestClassifier())

In [25]:
scores = cross_val_score(pipeline, sentences, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.7843333333333333, 0.00806914562460681)